###ส่วนที่ 1 ดึงข้อมูลเว็บ ที่เก็บข้อมูลแคลเลอลี่ของ อาหาร ขนม เครื่องดื่มต่างๆ

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "http://event.sanook.com/health/calories/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
tables = soup.find_all('table')

data = []
table1 = tables[0]
rows = table1.find_all('tr')

for row in rows[1:]:
    cols = row.find_all('td')
    name = cols[0].get_text(strip=True)
    quantity = cols[1].get_text(strip=True)
    calories = cols[2].get_text(strip=True)
    data.append({ "ชื่อ": name, "จํานวน": quantity, "ปริมาณแคลเลอรี่": calories,"Category": "เครื่องดื่ม"})

table2 = tables[1]
rows = table2.find_all('tr')

for row in rows[1:]:
    cols = row.find_all('td')
    name = cols[0].get_text(strip=True)
    quantity = cols[1].get_text(strip=True)
    calories = cols[2].get_text(strip=True)
    data.append({ "ชื่อ": name, "จํานวน": quantity, "ปริมาณแคลเลอรี่": calories,"Category": "ผลไม้ ของกินเล่น"})

table3 = tables[2]
rows = table3.find_all('tr')

for row in rows[1:]:
    cols = row.find_all('td')
    name = cols[0].get_text(strip=True)
    quantity = cols[1].get_text(strip=True)
    calories = cols[2].get_text(strip=True)
    data.append({ "ชื่อ": name, "จํานวน": quantity, "ปริมาณแคลเลอรี่": calories,"Category": "อาหาร"})

df = pd.DataFrame(data)
df['ปริมาณแคลเลอรี่'] = df['ปริมาณแคลเลอรี่'].str.replace(r'[^0-9]', '', regex=True)
df


,ชื่อ,จํานวน,ปริมาณแคลเลอรี่,Category
0,น้ำกระเจี๊ยบ,1 แก้ว,120,เครื่องดื่ม
1,"น้ำโค้ก, น้ำเป๊ปซี่",1 แก้ว,100,เครื่องดื่ม
2,น้ำมะเขือเทศ,1 แก้ว,48,เครื่องดื่ม
3,น้ำมะตูม,1 แก้ว,120,เครื่องดื่ม
4,น้ำมะนาว,1 แก้ว,100,เครื่องดื่ม
...,...,...,...,...
667,ไอศกรีมกาแฟ,1 ก้อน,142,ผลไม้ ของกินเล่น
668,ไอศกรีมชอกโกแล็ต,1 ก้อน,110,ผลไม้ ของกินเล่น
669,ไอศกรีมวนิลลา,1 ก้อน,140,ผลไม้ ของกินเล่น
670,ไอศกรีมสตรอเบอรี่,1 ก้อน,110,ผลไม้ ของกินเล่น


In [ ]:
df.to_csv('/content/drive/MyDrive/2567Colab statCamp/calories_food.csv', index=False)

ส่วนที่2 ดึงข้อมูลวัตถุดิบ เพื่อเช็ค แคลเลอลี่ โปรตีน คาร์โบไฮเดต ไขมัน

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_carbohydrates_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    rows = table.find_all("tr")
    data = []
    for row in rows[1:]:  # ข้ามแถวหัวตารางไป
        cols = row.find_all("td")  # หาคอลัมน์ในแถว
        if len(cols) == 5:  # ถ้าจำนวนคอลัมน์ในแถวคือ 5 (ข้อมูลครบ)
            food_type = cols[0].get_text(strip=True)  # ดึงชื่อประเภทอาหาร
            energy = cols[1].get_text(strip=True)  # ดึงค่าพลังงาน
            protein = cols[2].get_text(strip=True)  # ดึงปริมาณโปรตีน
            fat = cols[3].get_text(strip=True)  # ดึงปริมาณไขมัน
            carbohydrates = cols[4].get_text(strip=True)  # ดึงปริมาณคาร์โบไฮเดรต
            data.append([food_type, energy, protein, fat, carbohydrates, "คาร์โบไฮเดรตหลัก"])  # เก็บข้อมูลและเพิ่มประเภทอาหารเป็น "คาร์โบไฮเดรตหลัก"

    df_carbohydrates = pd.DataFrame(data, columns=["วัตถุดิบอาหาร(100กรัม)", "พลังงาน", "โปรตีน", "ไขมัน", "คาร์โบไฮเดต", "ประเภทวัตถุดิบ"])
    return df_carbohydrates

# ฟังก์ชันสำหรับการดึงข้อมูลจากหน้าโปรตีน
def scrape_protein_page(url):
    response = requests.get(url)  # ส่งคำขอไปที่ URL
    soup = BeautifulSoup(response.content, "html.parser")  # แปลงเนื้อหาของเว็บให้เป็นรูปแบบที่เข้าใจได้

    # หาแท็กที่เป็นตารางที่มีข้อมูล
    table = soup.find("table")
    rows = table.find_all("tr")  # หาแถวทั้งหมดในตาราง

    # ดึงข้อมูลที่ต้องการ เช่น ประเภทอาหาร, พลังงาน, โปรตีน, ไขมัน, คาร์โบไฮเดรต
    data = []
    for row in rows[1:]:  # ข้ามแถวหัวตารางไป
        cols = row.find_all("td")  # หาคอลัมน์ในแถว
        if len(cols) == 5:  # ถ้าจำนวนคอลัมน์ในแถวคือ 5 (ข้อมูลครบ)
            food_type = cols[0].get_text(strip=True)  # ดึงชื่อประเภทอาหาร
            energy = cols[1].get_text(strip=True)  # ดึงค่าพลังงาน
            protein = cols[2].get_text(strip=True)  # ดึงปริมาณโปรตีน
            fat = cols[3].get_text(strip=True)  # ดึงปริมาณไขมัน
            carbohydrates = cols[4].get_text(strip=True)  # ดึงปริมาณคาร์โบไฮเดรต
            data.append([food_type, energy, protein, fat, carbohydrates, "โปรตีนหลัก"])  # เก็บข้อมูลและเพิ่มประเภทอาหารเป็น "โปรตีนหลัก"

    # เปลี่ยนข้อมูลที่ดึงมาเป็น DataFrame (ตารางข้อมูลใน pandas)
    df_protein = pd.DataFrame(data, columns=["วัตถุดิบอาหาร(100กรัม)", "พลังงาน", "โปรตีน", "ไขมัน", "คาร์โบไฮเดต", "ประเภทวัตถุดิบ"])
    return df_protein

# ฟังก์ชันสำหรับการดึงข้อมูลจากหน้าไขมัน
def scrape_fat_page(url):
    response = requests.get(url)  # ส่งคำขอไปที่ URL
    soup = BeautifulSoup(response.content, "html.parser")  # แปลงเนื้อหาของเว็บให้เป็นรูปแบบที่เข้าใจได้

    # หาแท็กที่เป็นตารางที่มีข้อมูล
    table = soup.find("table")
    rows = table.find_all("tr")  # หาแถวทั้งหมดในตาราง
    data = []
    for row in rows[1:]:  # ข้ามแถวหัวตารางไป
        cols = row.find_all("td")
        if len(cols) == 5:
            food_type = cols[0].get_text(strip=True)
            energy = cols[1].get_text(strip=True)
            protein = cols[2].get_text(strip=True)
            fat = cols[3].get_text(strip=True)
            carbohydrates = cols[4].get_text(strip=True)  #
            data.append([food_type, energy, protein, fat, carbohydrates, "ไขมันหลัก"])

    # เปลี่ยนข้อมูลที่ดึงมาเป็น DataFrame (ตารางข้อมูลใน pandas)
    df_fat = pd.DataFrame(data, columns=["วัตถุดิบอาหาร(100กรัม)", "พลังงาน", "โปรตีน", "ไขมัน", "คาร์โบไฮเดต", "ประเภทวัตถุดิบ"])
    return df_fat

# URLs ของหน้าเว็บ
url_carbohydrates = "https://www.thaisook.org/2023/09/14/carbohydrate/"
url_protein = "https://www.thaisook.org/2023/09/12/protein-2/"
url_fat = "https://www.thaisook.org/2023/09/19/fat/"

df_carbohydrates = scrape_carbohydrates_page(url_carbohydrates)
df_protein = scrape_protein_page(url_protein)
df_fat = scrape_fat_page(url_fat)
df_combined = pd.concat([df_carbohydrates, df_protein, df_fat], ignore_index=True)
df_combined



,วัตถุดิบอาหาร(100กรัม),พลังงาน,โปรตีน,ไขมัน,คาร์โบไฮเดต,ประเภทวัตถุดิบ
0,ข้าวกล้อง,111,2.6,0.9,23,คาร์โบไฮเดรตหลัก
1,ข้าวไรส์เบอรรี่,111,2.6,0.9,23,คาร์โบไฮเดรตหลัก
2,ข้าวขาว,130,2.7,0.3,28,คาร์โบไฮเดรตหลัก
3,ขนมปัง,361,12,1.7,72.5,คาร์โบไฮเดรตหลัก
4,มันเทศ,97,2,1,20,คาร์โบไฮเดรตหลัก
5,เผือก,90,2,0.2,20.7,คาร์โบไฮเดรตหลัก
6,ฟักทอง,26,1,0.1,6.5,คาร์โบไฮเดรตหลัก
7,เส้นเล็ก,108,1.8,0.2,24,คาร์โบไฮเดรตหลัก
8,วุ้นเส้น,351,0.2,0.1,86.1,คาร์โบไฮเดรตหลัก
9,ข้าวเหนียว,97,2,0.2,21.1,คาร์โบไฮเดรตหลัก


In [ ]:
df_combined.to_csv('/content/drive/MyDrive/2567Colab statCamp/สาอาหารวัตถุดิบ1.csv', index=False)